In [1]:
import os

In [2]:
%pwd

'd:\\text summarisation\\Text-Summarizer-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\text summarisation\\Text-Summarizer-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [11]:
pip install python-box


  Obtaining dependency information for python-box from https://files.pythonhosted.org/packages/b8/0e/2aea4a9180e3cf58efe4119ce8c9f75ef97f2666bd1fe2b1b5cd8e36170c/python_box-7.1.1-cp38-cp38-win_amd64.whl.metadata
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   - -------------------------------------- 0.0/1.2 MB 262.6 kB/s eta 0:00:05
   - -------------------------------------- 0.0/1.2 MB 245.8 kB/s eta 0:00:05
   -- ------------------------------------- 0.1/1.2 MB 326.8 kB/s eta 0:00:04
   ---- ----------------------------------- 0.1/1.2 MB 450.6 kB/s eta 0:00:03
   ------ --------------------------------- 0.2/1.2 MB 620.6 kB/s eta 0:00:02
   --------- ------------------------------ 0.3/1.2 MB 739.6 kB/s eta 0:00:02
   ------------ --------------------------- 0.4/1.2 MB 890.4 kB/s eta 0:00:01
   ------------- ------------------

In [17]:
pip install ensure

  Using cached ensure-1.0.3-py3-none-any.whl (15 kB)
Note: you may need to restart the kernel to use updated packages.


In [18]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories

In [19]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [21]:
import os
import urllib.request as request
import zipfile
from src.textSummarizer.logging import logger
from src.textSummarizer.utils.common import get_size

In [22]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [23]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-10-05 20:32:22,252: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-10-05 20:32:22,262: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-05 20:32:22,262: INFO: common: created directory at: artifacts]
[2023-10-05 20:32:22,276: INFO: common: created directory at: artifacts/data_ingestion]


[2023-10-05 20:32:44,372: INFO: 1434958058: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 15729059
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "5c74167371e5fc4354cb3bca7c69abea73773bd9f24db75f159b8864a891e649"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 266C:39CE69:2C33C6:316B85:651ED000
Accept-Ranges: bytes
Date: Thu, 05 Oct 2023 15:02:26 GMT
Via: 1.1 varnish
X-Served-By: cache-bom4720-BOM
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1696518145.224819,VS0,VE1270
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 05dcbabc7d4631f0357314a836c6667f9225b483
Expires: Thu, 05 Oct 2023 15:07:26 GMT
Source-Age: 1

]
